# Downstream (MR)

Use a trained encoder checkpoint for MR classification.

In [ ]:
# Adjust paths before running
BASE_MODEL = "bert-base-uncased"
ENCODER_CHECKPOINT = "./checkpoints/demo_nli/bert-base-uncased_best.pkl"
DATA_DIR = "./data"
OUTPUT_DIR = "./checkpoints/demo_mr"


In [ ]:
# MR fine-tuning using the encoder checkpoint
!python training/mr/train_mr.py \
    {BASE_MODEL} \
    --encoder_checkpoint {ENCODER_CHECKPOINT} \
    --data_dir {DATA_DIR} \
    --output_dir {OUTPUT_DIR} \
    --epochs 1 \
    --batch_size 32 \
    --use_cuda
